In [ ]:
!git clone https://github.com/hahnicity/ventmode
!python3 -m pip install ./ventmode
!python -m pip install git+https://github.com/lukauskas/dtwco.git#egg=dtwco
!git clone https://github.com/Villiamfj/ventTools
%pip install -e ventTools tensorflow_decision_forests
%pip install -q -U keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import numpy as np
import ventTools
from ventTools.data import convert_to_fixed_length
from ast import literal_eval
import ventmode
from ventmode import datasets
from os.path import join
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Test Ventmode Dataset

In [ ]:
mode_to_index = {
    0: 0, 
    1: 1, 
    3: 2, 
    4: 3, 
    6: 4
}

vfinal_features = [
    #"itime_var_max_win",#
    #"maw_var",#
    "flow_var_var",
    "all_pressure_var",
    "flow_var",
    #"pip_min_peep",#
    #"tvi_var",#
    #"pressure_var",#
    "itime_var",
    "pressure_itime_var",
    "pressure_itime_var_max_win",
    "n_plats_past_20",
    #'median_peep',#
    #'median_pip',#
    #'ipauc:epauc',#
]

def test_ventmode_dataset(df, classifer, metrics, label='y'):
  test_df = df.copy()

  #test_feat = test_df[vfinal_features + [label]]
  test_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(test_feat, label = label)
  
  classifer.compile(metrics=metrics)
  classifer.evaluate(test_dataset)


def _get_df(fileset):
  vfinal = datasets.VFinalFeatureSet(fileset, 10, 100)
  df = vfinal.create_learning_df()

  df = df[df['y'].isin(mode_to_index.keys())]
  df['y'] = df['y'].apply(lambda x: mode_to_index[x])

  df = df.select_dtypes(include=[np.number])
  df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

  return df[df.columns.difference(['y', 'rel_bn', 'vent_bn', 'idx', 'x0_idx'])], df[['y']]


def get_train_df():
  COHORT_X_DIR = join("ventmode/anon_train_data", "raw_vwd")
  COHORT_Y_DIR = join("ventmode/anon_train_data", "y_dir")
  fileset = datasets.get_cohort_fileset(COHORT_X_DIR, COHORT_Y_DIR)
  return _get_df(fileset)


def get_test_df():
  TEST_COHORT_X_DIR = join("ventmode/anon_test_data", "raw_vwd")
  TEST_COHORT_Y_DIR = join("ventmode/anon_test_data", "y_dir")
  test_fileset = datasets.get_cohort_fileset(TEST_COHORT_X_DIR, TEST_COHORT_Y_DIR)
  return _get_df(test_fileset)


In [ ]:
X_train, y_train = get_train_df()
X_test, y_test = get_test_df()


X_all_copy, y_all_copy = X_train.copy(), y_train.copy()
X_test_all_copy, y_test_all_copy = X_test.copy(), y_test.copy()
# X_train = X_train[vfinal_features]
# X_test = X_test[vfinal_features]

X_copy, y_copy = X_train.copy(), y_train.copy()
X_test_copy, y_test_copy = X_test.copy(), y_test.copy()

In [ ]:


X_train, y_train = X_copy.copy(), y_copy.copy()
X_test, y_test = X_test_copy.copy(), y_test_copy.copy()


scaler = RobustScaler()
cols = X_train.columns
train_index = X_train.index
test_index = X_test.index
X_train = scaler.fit_transform(X_train)
if len(X_test) != 0:
    X_test = scaler.transform(X_test)
X_train, X_test = pd.DataFrame(X_train, index=train_index, columns=cols), pd.DataFrame(X_test, index=test_index, columns=cols)

X_copy, y_copy = X_train.copy(), y_train.copy()
X_test_copy, y_test_copy = X_test.copy(), y_test.copy()

In [ ]:
def evaluate(y_true, y_pred):
  print(classification_report(y_true, y_pred))
  print(confusion_matrix(y_true, y_pred))

In [ ]:

X_train, y_train = X_copy.copy(), y_copy.copy()
X_test, y_test = X_test_copy.copy(), y_test_copy.copy()


clf = LinearSVC(C=10)
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

In [ ]:
from sklearn.neural_network import MLPClassifier


X_train, y_train = X_copy.copy(), y_copy.copy()
X_test, y_test = X_test_copy.copy(), y_test_copy.copy()


clf = MLPClassifier(activation='tanh', learning_rate_init = 0.01, solver = 'adam', hidden_layer_sizes = [64, 32])
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

In [ ]:

import tensorflow_decision_forests as tfdf

X_train, y_train = X_copy.copy(), y_copy.copy()
X_test, y_test = X_test_copy.copy(), y_test_copy.copy()

clf = RandomForestClassifier(max_features='auto', n_estimators=30, max_depth=15, criterion='entropy', random_state=1)
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

In [ ]:

X_train, y_train = X_copy.copy(), y_copy.copy()
X_test, y_test = X_test_copy.copy(), y_test_copy.copy()

clf = LogisticRegression(penalty='l2', C=4,tol=.0001, solver='lbfgs')
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

Finding target length

In [ ]:
X_train.shape

In [ ]:
percentages = [0.1, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.9, 0.95]
quantiles = X_train['breath_time'].quantile(percentages)
quantiles / 0.02

In [ ]:
samples = X_train['breath_time'] / 0.02
samples.hist(bins=20)